In [1]:
from datetime import datetime,date
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import re

In [2]:
# -------------------------------------------
# Modify the holiday class to 
# 1. Only accept Datetime objects for date.
# 2. You may need to add additional functions
# 3. You may drop the init if you are using @dataclasses
# --------------------------------------------
@dataclass
class Holiday:
    name: str
    date: datetime
    
    def __str__ (self):
        return self.name
        # String output
        # Holiday output when printed.

In [13]:
def getHTML(url):
    response = requests.get(url)
    return response.text

html = requests.get("https://www.timeanddate.com/holidays/us/").text
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', attrs={'id':'holidays-table'})
tbody = table.find('tbody')

In [ ]:
e1 = tbody.find('th', attrs={'class':'nw'})

In [ ]:
print(example)

In [ ]:
example.text

In [ ]:
e2 = tbody.find('a')

In [ ]:
type(date(2020,1,3))

In [ ]:
datetime.strptime("Jan 1 2020", "%b %d %Y")

In [ ]:
holidays = []

In [ ]:
holiday_list = []
year = 2022
for date_, holiday_ in zip(tbody.find_all('th'),tbody.find_all('a')):
    fDate = datetime.strptime((date_.text + " " + str(year)),"%b %d %Y").date()
    temp_holiday = Holiday(date = fDate,name = holiday_.text)
    if temp_holiday not in holiday_list:
        holiday_list.append(temp_holiday)

In [ ]:
for row in tbody.find_all('a'):
    print(row.text)

In [50]:
# -------------------------------------------
# The HolidayList class acts as a wrapper and container
# For the list of holidays
# Each method has pseudo-code instructions
# --------------------------------------------
@dataclass
class HolidayList:
    innerHolidays: list[Holiday] = field(default_factory=list)
   
    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            self.innerHolidays.append(holidayObj)
        else:
            print(f"{holidayObj} is not a Holiday Object")
        # Make sure holidayObj is an Holiday Object by checking the type
        # Use innerHolidays.append(holidayObj) to add holiday
        # print to the user that you added a holiday

    def findHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            if HolidayName == holiday.name and Date == holiday.date:
                return holiday
        print(f'{HolidayName} and {Date} not in list of holidays')
        return None
        # Find Holiday in innerHolidays
        # Return Holiday

    def removeHoliday(self, HolidayName, Date):
        for i in range(len(self.innerHolidays)):
            if HolidayName == self.innerHolidays[i].name and Date == self.innerHolidays[i].date:
                removed = self.innerHolidays.pop(i)
                print(f"{removed} has been removed from the list")
        # Find Holiday in innerHolidays by searching the name and date combination.
        # remove the Holiday from innerHolidays
        # inform user you deleted the holiday

    def read_json(self, filelocation):
        with open(filelocation, 'r') as f:
            data = json.load(f)
        for hol in data['holidays']:
            tmpDate = datetime.strptime(hol['date'],"%Y-%m-%d").date()
            tmpName = hol['name']
            tmpHoliday = Holiday(tmpName,tmpDate)
            self.addHoliday(tmpHoliday)
        # Read in things from json file location
        # Use addHoliday function to add holidays to inner list.

    def save_to_json(self, filelocation):
        with open(filelocation, 'w') as f:
            data = json.dump(self.innerHolidays, f)
        # Write out json file to selected file.
        
    def scrapeHolidays(self):
        for year in range(2020,2025):
            html = requests.get(f"https://www.timeanddate.com/holidays/us/{year}").text
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find('table', attrs={'id':'holidays-table'})
            tbody = table.find('tbody')
            for date_, holiday_ in zip(tbody.find_all('th'),tbody.find_all('a')):
                fDate = datetime.strptime((date_.text + " " + str(year)),"%b %d %Y").date()
                temp_holiday = Holiday(date = fDate,name = holiday_.text)
                if temp_holiday not in self.innerHolidays:
                    self.addHoliday(temp_holiday)
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
        # Remember, 2 previous years, current year, and 2  years into the future. You can scrape multiple years by adding year to the timeanddate URL. For example https://www.timeanddate.com/holidays/us/2022
        # Check to see if name and date of holiday is in innerHolidays array
        # Add non-duplicates to innerHolidays
        # Handle any exceptions.     

    def numHolidays(self):
        return len(self.innerHolidays)
        # Return the total number of holidays in innerHolidays
    
    def filter_holidays_by_week(self, year, week_number):
        pass
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        # Week number is part of the the Datetime object
        # Cast filter results as list
        # return your holidays

    def displayHolidaysInWeek(self, holidayList):
        pass
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        # Output formated holidays in the week. 
        # * Remember to use the holiday __str__ method.

    def getWeather(self, weekNum):
        pass
        # Convert weekNum to range between two days
        # Use Try / Except to catch problems
        # Query API for weather in that week range
        # Format weather information and return weather string.

    def viewCurrentWeek(self):
        pass
        # Use the Datetime Module to look up current week and year
        # Use your filter_holidays_by_week function to get the list of holidays 
        # for the current week/year
        # Use your displayHolidaysInWeek function to display the holidays in the week
        # Ask user if they want to get the weather
        # If yes, use your getWeather function and display results

In [ ]:
def filter(h, year, week):
    

In [42]:
h = HolidayList()

In [43]:
h.read_json('holidays.json')

In [38]:
h.numHolidays()

7

In [44]:
h.scrapeHolidays()

In [45]:
h.numHolidays()

2638

In [34]:
len([hol for hol in h.innerHolidays if hol.date.year == 2024])

513

In [32]:
for hol in h.innerHolidays:
    if hol.date.year == 2024:
        print(hol,hol.date)

New Year's Day 2024-01-01
New Year's Eve observed 2024-01-01
World Braille Day 2024-01-04
Epiphany 2024-01-06
Orthodox Christmas Day 2024-01-07
International Programmers' Day 2024-01-07
Estelle Reel Day 2024-01-07
Battle of New Orleans 2024-01-08
Stephen Foster Memorial Day 2024-01-13
Orthodox New Year 2024-01-14
Martin Luther King Jr. Day 2024-01-15
Robert E. Lee's Birthday 2024-01-15
Idaho Human Rights Day 2024-01-15
Civil Rights Day 2024-01-15
Robert E. Lee's Birthday 2024-01-19
State Holiday 2024-01-19
Confederate Heroes' Day 2024-01-19
World Religion Day 2024-01-21
International Day of Education 2024-01-24
Tu Bishvat/Tu B'Shevat 2024-01-25
International Customs Day 2024-01-26
International Day of Commemoration in Memory of the Victims of the Holocaust 2024-01-27
World Leprosy Day 2024-01-28
Kansas Day 2024-01-29
National Freedom Day 2024-02-01
First Day of Black History Month 2024-02-01
World Wetlands Day 2024-02-02
Groundhog Day 2024-02-02
National Wear Red Day 2024-02-02
Interna

In [ ]:
def save_to_json(h, filelocation):

    with open(filelocation, 'w') as f:
        data = json.dump(self.innerHoliday, f)
    # Write out json file to selected file.

In [4]:
def read_json(holidays, file):    
    with open(file, 'r') as f:
        data = json.load(f)
    for hol in data['holidays']:
        tmpDate = datetime.strptime(hol['date'],"%Y-%m-%d").date()
        tmpName = hol['name']
        tmpHoliday = Holiday(tmpName,tmpDate)
        # print((data['holidays'][0]))

In [5]:
h = []
read_json(h, 'holidays.json')

{'name': 'Margaret Thatcher Day', 'date': '2021-01-10'}
Margaret Thatcher Day
{'name': 'World Sketchnote Day', 'date': '2021-01-11'}
World Sketchnote Day
{'name': 'Zanzibar Revolution Day', 'date': '2021-01-12'}
Zanzibar Revolution Day
{'name': 'National Rubber Ducky Day', 'date': '2021-01-13'}
National Rubber Ducky Day
{'name': 'Tamil Thai Pongdal Day', 'date': '2021-01-14'}
Tamil Thai Pongdal Day
{'name': 'National Bagel Day', 'date': '2021-01-15'}
National Bagel Day
{'name': 'Signing of the Peace Accords', 'date': '2021-01-16'}
Signing of the Peace Accords


In [7]:
h = HolidayList()

In [27]:
h10 = Holiday('myBirthday',date(1999,12,1))

In [30]:
type(h10.date.year)

int

In [9]:
h.addHoliday(h1)

In [10]:
print(h.innerHolidays)

[Holiday(name='myBirthday', date=datetime.date(1999, 12, 1))]


In [ ]:
fDate = datetime.strptime((hol['date'],"%Yb-%m-%d").date()

In [ ]:
data = [
    {'name':'Will','age':55},
    {'name':'Heather','age':54}
]

In [ ]:
data.sort(key=lambda item: item['name'])
data

In [ ]:
def main():
    pass
    # Large Pseudo Code steps
    # -------------------------------------
    # 1. Initialize HolidayList Object
    # 2. Load JSON file via HolidayList read_json function
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    # 3. Create while loop for user to keep adding or working with the Calender
    # 4. Display User Menu (Print the menu)
    # 5. Take user input for their action based on Menu and check the user input for errors
    # 6. Run appropriate method from the HolidayList object depending on what the user input is
    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 


In [ ]:
main()